# Code to replicate Figures (a,e)

# Installing packages

In [1]:
!git clone --recursive https://github.com/oseledets/ttpy.git
!python ttpy/setup.py install --quiet



Cloning into 'ttpy'...
remote: Enumerating objects: 2128, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 2128 (delta 9), reused 17 (delta 5), pack-reused 2100 (from 1)
Receiving objects: 100% (2128/2128), 715.44 KiB | 14.31 MiB/s, done.
Resolving deltas: 100% (1381/1381), done.
Submodule 'tt/cross/rectcross' (https://bitbucket.org/oseledets/rectcross) registered for path 'tt/cross/rectcross'
Submodule 'tt/tt-fort' (https://github.com/oseledets/tt-fort.git) registered for path 'tt/tt-fort'
Submodule 'tt/utils/rect_maxvol' (https://bitbucket.org/muxas/rect_maxvol) registered for path 'tt/utils/rect_maxvol'
Cloning into '/content/ttpy/tt/cross/rectcross'...
remote: Enumerating objects: 112, done.        
remote: Counting objects: 100% (112/112), done.        
remote: Compressing objects: 100% (107/107), done.        
remote: Total 112 (delta 56), reused 0 (delta 0), pack-reused 0 (from 0)        
Receiving objects: 100% (

# Importing packages

In [2]:
import numpy as np
from scipy.optimize import minimize
import time
import statistics
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [3]:
import tt
from tt.core.vector import vector

# Required Functions

## Basic functions

In [9]:
# Function to convert a number to any base expansion
# x: number
# bitNo; number of bits in the expansion
# base: base

def basary(x, bitno, base):
  ans_list = []
  q = np.copy(x)
  while q != 0:
    r = q % base
    q = q // base
    ans_list.append(r)
  ans_list = list(reversed(ans_list))
  if len(ans_list) < bitno:
    ans_list = [0] * (bitno - len(ans_list)) + ans_list
  return ans_list

# This function returns a numpy matrix of the form R_Z(theta) for any real number theta
def R_Z(theta):
  return np.matrix([[np.exp(-1j * theta/2), 0],[0, np.exp(1j * theta/2)]])

# This function returns a numpy matrix of the form R_Y(theta) for any real number theta
def R_Y(theta):
  return np.matrix([[np.cos(theta/2), (-1 * np.sin(theta/2))], [np.sin(theta/2), np.cos(theta/2)]])

# This function returns a numpy matrix of the form R_X(theta) for any real number theta
def R_X(theta):
  return np.matrix([[np.cos(theta/2), ((-1j) * np.sin(theta/2))], [((-1j) * np.sin(theta/2)), np.cos(theta/2)]])

CNOT_12 = np.array([[1 + 0j, 0 + 0j, 0 + 0j, 0 + 0j],
                  [0 + 0j, 1 + 0j, 0 + 0j, 0 + 0j],
                  [0 + 0j, 0 + 0j, 0 + 0j, 1 + 0j],
                  [0 + 0j, 0 + 0j, 1 + 0j, 0 + 0j]])

CNOT_21 = np.array([[1 + 0j, 0 + 0j, 0 + 0j, 0 + 0j],
                  [0 + 0j, 0 + 0j, 0 + 0j, 1 + 0j],
                  [0 + 0j, 0 + 0j, 1 + 0j, 0 + 0j],
                  [0 + 0j, 1 + 0j, 0 + 0j, 0 + 0j]])

SWAP = np.matrix([[1,0,0,0],
                 [0,0,1,0],
                 [0,1,0,0],
                 [0,0,0,1]])

ket_0 = np.array([[1,0],
                  [0,0]])
def CNOT(is_rev=0):
  if is_rev == 0:
    return CNOT_12

  else:
    return CNOT_21

Id4 = np.eye(4)

# This function returns a matrix of the subcircuit s_gate.
# 1, theta: A 2 dimensional numpy array
def s_gate(theta):
  return np.kron(R_X(theta[0]), R_Y(theta[1])) @ CNOT()


# Inverse of above gate
def s_gate_inverse(theta):
  return CNOT() @ np.kron(R_X(-theta[0]), R_Y(-theta[1]))

# Above gate with double the depth
def s2_gate_inverse(theta):
  return CNOT() @ np.kron(R_X(-theta[0]), R_Y(-theta[1])) @ CNOT() @ np.kron(R_X(-theta[2]), R_Y(-theta[3]))

# Function to apply a gate of a state
# state: an n-dimensional array representing a state or a list of core tensors representing the state given as MPS
# qubits_to_act: the qubits to act the gate on
# gate: gate
def apply_gate(state, qubits_to_act, gate):
  temp_state = np.copy(state)
  temp_state = np.tensordot(temp_state, np.reshape(np.array(gate), tuple([temp_state.shape[0]] * (2 * len(qubits_to_act)))), axes = [qubits_to_act, list(range(-len(qubits_to_act),0))])
  temp_state = np.moveaxis(temp_state, list(range(-len(qubits_to_act),0)), qubits_to_act)
  return temp_state

# Function that applies the inverse of an MPS ansatz
# state: input mps
# theta: vector of parameters
# step_size: subcircuit width
def mps_inverse(state, theta, step_size):
  temp_state = list.copy(state)
  n_qubits = len(state)
  len_s = 2
  pos = 0
  gate_list = []

  r = 0
  for i in range(n_qubits - step_size + 1):
    for d in range(step_size):
      for j in range(i + (d%2), i + step_size, 2):
        if (j + 1) < (i + step_size):
          if step_size != 2:
            gate = s_gate_inverse(theta[pos * len_s: (pos+1) * len_s])
            #print([j, j+1])
            gate_list.append((gate, [j, j+1]))
            pos += 1
          else:
            #print("YES")
            gate = s2_gate_inverse(theta[2 * pos * len_s: 2 * (pos+1) * len_s])
            #print([j, j+1])
            gate_list.append((gate, [j, j+1]))
            pos += 1


  for i in range(len(gate_list)-1, -1, -1):
  #for i in range(len(gate_list)):
    if len(gate_list[i][1]) == 2:
      #print(gate_list[i][1])
      temp_state = apply_two_qubit_gate(temp_state, gate_list[i][0], gate_list[i][1])
      #temp_state = apply_two_qubit_gate(temp_state, np.conj(gate_list[i][0]).T, gate_list[i][1])
      temp_state = round_mps(temp_state, 0.000000001)
    else:
      #print(gate_list[i][1])
      temp_state = apply_multi_qubit_gate(temp_state, gate_list[i][0], gate_list[i][1])
      temp_state = round_mps(temp_state, 0.000000001)
  return temp_state

# Function that initializes the parameter vector
def initialize_random_theta(n_qubits, step_size=0, epsilon = 2*np.pi):
  if step_size == 2:
    #step_size = int(np.log2(n_qubits))
    theta_len = 4 * (n_qubits-step_size+1)
    theta = np.random.uniform(0, epsilon, theta_len)
  elif step_size == 3:
    theta_len = 6 * (n_qubits-step_size+1)
    theta = np.random.uniform(0, epsilon, theta_len)
  elif step_size == 4:
    theta_len = 12 * (n_qubits-step_size+1)
    theta = np.random.uniform(0, epsilon, theta_len)
  return theta

# Function to implement a single update of the spsa algorithm
# state: input mps
# c_alpha: spsa parameter
# a_alpha: spsa parameter
# idx: current iteration
# measurement_strategy: "local" or "global"
# step_size: subcircuit width
def single_spsa_update(state, theta, c_alpha, a_alpha, idx, measurement_strategy, step_size = 0):
  temp_state = list.copy(state)
  n_qubits = len(state)
  #print('n_qubits: ', n_qubits)
  delta = np.random.choice([-1, 1], len(theta), p = [1/2] * 2)
  #step_size = int(np.log2(n_qubits))
  c = 1/(idx ** c_alpha)
  a = 1/(idx ** a_alpha)

  state_plus = mps_inverse(temp_state, theta + (c * delta), step_size)
  state_minus = mps_inverse(temp_state, theta - (c * delta), step_size)

  if measurement_strategy == 'local':
    g = (measure_individual_qubits_mps(state_plus) - measure_individual_qubits_mps(state_minus))/(2 * c)

  elif measurement_strategy == 'global':
    g = ((np.abs(query_mps(state_plus, 0)) ** 2) - (np.abs(query_mps(state_minus, 0)) ** 2))/(2 * c)

  return theta + (a * g * delta)

# Function to implement the whole spsa optimiztion
# theta: initial guess of parameters
# state: input mps
# c_alpha: spsa parameter
# a_alpha: spsa parameter
# no_of_iter: total number of iterations
# measurement_strategy: "local" or "global"
# step_size: subcircuit width
# interval: interval points to compute infidelities throughout the optimization
def spsa(theta, state, c_alpha, a_alpha, no_of_iter, measurement_strategy, step_size = 0, interval = 100):
    iter_no = 1
    n_qubits = len(state)
    infid_list = []
    theta_list = []

    while iter_no < no_of_iter + 1:

        theta = single_spsa_update(state, theta, c_alpha, a_alpha, iter_no, measurement_strategy, step_size)
        if iter_no % interval == 0:
            init_state = list.copy(state)

            init_state = mps_inverse(init_state, theta, step_size)

            fidelity = np.abs(query_mps(init_state, 0)) ** 2

            infid_list.append(1-fidelity)
            theta_list.append(theta)
        if iter_no % 100 == 0:
            print("Current iteration: {}".format(iter_no))
            print("Current Infidelity: {}".format(1-fidelity))
        iter_no += 1
    return infid_list, theta_list



## MPS related

In [5]:
# Function to apply a single qubit gate to an mps
# mps: mps given as a list of core tensors
# gate: gate
# qubit: qubit index
def apply_single_qubit_gate(mps, gate, qubit):
  new_mps = list.copy(mps)
  temp = np.tensordot(gate, new_mps[qubit], axes=([1], [0]))
  new_mps[qubit] = np.copy(temp)
  return new_mps

# Function to apply a two qubit gate to an mps
# mps: mps given as a list of core tensors
# gate: gate
# qubits: list of two qubit indices
def apply_two_qubit_gate(mps, gate, qubits):
  dim = mps[qubits[0]].shape[0]
  if np.abs(qubits[0] - qubits[1]) == 1:

    chi1 = mps[qubits[0]].shape[1]
    chi2 = mps[qubits[1]].shape[2]
    gate = np.reshape(np.array(gate), (dim,dim,dim,dim))
    temp = np.tensordot(gate, mps[qubits[0]], axes=([2], [0]))
    temp = np.tensordot(temp, mps[qubits[1]], axes=([2, 4], [0, 1]))
    temp = np.ndarray.transpose(temp, [0,2,1,3])
    temp = np.reshape(temp, (dim * chi1, dim * chi2))
    u, sigma, vh = np.linalg.svd(temp, full_matrices = False)
    u = u @ np.diag(sigma)
    u = np.reshape(u, (dim, chi1, len(sigma)))
    vh = np.reshape(vh, (len(sigma), dim, chi2))
    vh = np.ndarray.transpose(vh, [1,0,2])
    mps[qubits[0]] = u
    mps[qubits[1]] = vh
    return mps
  else:
    l = min(qubits)
    m = max(qubits)
    temp_mps = list.copy(mps)
    for i in range(l, m-1):
      #print("Swapping ", [i, i+1])
      temp_mps = apply_two_qubit_gate(temp_mps, SWAP, [i, i+1])
    #print('m', m)
    if qubits[0] < qubits[1]:
      #print("Applying ", [m-1, m])
      temp_mps = apply_two_qubit_gate(temp_mps, gate, [m-1, m])

    for i in range(m-2, l-1, -1):
      #print("Swapping ", [i, i+1])
      temp_mps = apply_two_qubit_gate(temp_mps, SWAP, [i, i+1])
    return temp_mps

# Function to compute the MPS representation of a gate
def convert_gate_to_mps_actual(gate):
  n_qubits = int(len(gate.shape)/2)
  mps = []
  sigma_prev = 1
  dim = gate.shape[0]
  indices = []
  for i in range(n_qubits):
    indices.append(i)
    indices.append(i + n_qubits)
  gate = np.ndarray.transpose(gate, indices)

  gate = np.reshape(gate, tuple([dim * dim] * n_qubits))
  gate = np.reshape(gate, (dim * dim, (dim * dim) ** (n_qubits-1)))
  for i in range(n_qubits):
    gate = np.reshape(gate, (dim * dim * (sigma_prev), int((gate.shape[1] * gate.shape[0])/(dim * dim * (sigma_prev)))))
    u, sigma, vh = np.linalg.svd(gate, full_matrices = False)
    u = u @ np.diag(sigma)
    u = np.reshape(u, (dim, dim, sigma_prev, len(sigma)))
    if i != n_qubits - 1:
      vh = np.reshape(vh, (len(sigma), dim * dim, int(vh.shape[0] * vh.shape[1]/(dim * dim * len(sigma)))))
      vh = np.ndarray.transpose(vh, [1,0,2])
      vh = np.reshape(vh, (vh.shape[0] * vh.shape[1], vh.shape[2]))
    gate = np.copy(vh)
    sigma_prev = np.copy(len(sigma))
    mps.append(u)
  return mps

# Function to apply a gate on a number of qubits greater than 2
# qubits: list of qubits
def apply_multi_qubit_gate(mps, gate, qubits):
  #mps = list.copy(temp_mps)
  dim = mps[0].shape[0]
  gate = np.reshape(gate, tuple([dim] * (2 * len(qubits))))
  gate_mps = convert_gate_to_mps_actual(gate)
  gate_mps_list = []
  starting_len = mps[qubits[0]].shape[1]
  for q in range(len(qubits)):
    temp = np.tensordot(gate_mps[q], mps[qubits[q]], [[1], [0]])
    temp = np.ndarray.transpose(temp, [0,1,3,2,4])
    temp = np.reshape(temp, (temp.shape[0], temp.shape[1] * temp.shape[2], temp.shape[3] * temp.shape[4]))

    gate_mps_list.append(temp)
  return mps[:qubits[0]] + gate_mps_list + mps[qubits[-1] + 1:]

# Function to compute the inner product between two MPSs
def inner_product(mps1, mps2):

    for i in range(len(mps1)):
        for j in range(mps1[i].shape[0]):
            temp = np.kron(mps1[i][j,:,:], np.conj(mps2[i][j,:,:]))
            if j == 0:
                A = temp
            else:
                A += temp

        if i == 0:
            ans_mat = A

        else:
            ans_mat = ans_mat @ A
    #print(ans_mat)
    return np.trace(ans_mat)

# Function to query the element in an index of an MPS
def query_mps(mps, index):
    n_qubits = len(mps)
    ans_list = []
    #q = np.copy(index)
    #ans = np.matrix([[1]])
    dim = mps[0].shape[0]
    #bin_list = binary(index, n_qubits)
    if type(index) != list:
      bin_list = basary(index, n_qubits, dim)

    else:
      bin_list = list.copy(index)
    for div in range(n_qubits):
        if div == 0:
          ans = mps[div][bin_list[div]]
        else:
          ans = ans @ mps[div][bin_list[div]]
    return ans[0,0]

# Function to compute the expectation of O.
def measure_individual_qubits_mps(mps):
  temp_mps = list.copy(mps)
  n_qubits = len(mps)
  ans = 0
  for i in range(n_qubits):
     #print('i: ', i)
     ans += inner_product(round_mps(apply_single_qubit_gate(temp_mps, ket_0, i), 0.000000001), mps)/n_qubits
  return ans

# Function that creates the MPS representation of the all0 state
def create_all0_state_mps(n_qubits):
  return [np.reshape(np.array([1,0]), (2,1,1))] * n_qubits




## Rounding

In [7]:
def change_tensor_order(mps):
    m = []
    for i in range(len(mps)):
        m.append(np.ndarray.transpose(mps[i], [1, 0, 2]))

    return m

def round_mps(mps, eps):
  v = vector()
  a = v.from_list(change_tensor_order(mps))
  ans_mps = change_tensor_order(v.to_list(a.round(eps)))
  return ans_mps


# Testing

In [10]:
n_qubits = 20
true_state = create_all0_state_mps(n_qubits) # target state


init_theta = initialize_random_theta(n_qubits, step_size = 2, epsilon = 1.1) # initialize theta
measurement_strategy = 'local'
c_alpha = 0.4
a_alpha = 0.4
no_of_iter = 2000
infid_list, theta_list = spsa(init_theta, true_state, c_alpha, a_alpha, no_of_iter, measurement_strategy, step_size = 2, interval = 100)

Current iteration: 100
Current Infidelity: 0.9927436433484945
Current iteration: 200
Current Infidelity: 0.866316649573202
Current iteration: 300
Current Infidelity: 0.6569006147733482
Current iteration: 400
Current Infidelity: 0.49247001746641106


KeyboardInterrupt: 